In [160]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime, timedelta

In [161]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['BOF2024_crawling']
collection = db['naver_cafe_crawling']

In [162]:
# 웹드라이버 초기화 및 네이버 카페 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://section.cafe.naver.com/")
driver.maximize_window()

# 3초간 대기
time.sleep(3)

In [163]:
# 검색어 입력 요소 찾기
search_box = driver.find_element(By.XPATH, '//input[@placeholder="원하는 카페, 글을 찾아보세요"]')

# 검색어 입력
search_box.send_keys('부산원아시아페스티벌')

# 3초간 대기 후 엔터 키 입력
time.sleep(3)
search_box.send_keys(Keys.ENTER)

In [164]:
try:
    SCROLL_PAUSE_TIME = 1.5
    current_page = 1
    all_data = []

    while True:
        # 페이지 스크롤
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        # 게시글 요소들 찾기
        elements = driver.find_elements(By.CSS_SELECTOR, 'div.ArticleItem')

        for element in elements:
            try:
                # strong 태그의 클래스명이 'title'인 요소 찾기
                title_element = element.find_element(By.CSS_SELECTOR, 'strong.title')
                title = title_element.text

                # a 태그의 href 속성값 가져오기
                url_element = element.find_element(By.CSS_SELECTOR, 'div.article_item_wrap > a')
                url = url_element.get_attribute('href')

                # span 태그의 클래스명이 'cafe_name'인 요소들 찾기
                cafe_name_element = element.find_element(By.CSS_SELECTOR, 'span.cafe_name')
                cafe_name = cafe_name_element.text

                # p 태그의 클래스명이 'text'인 요소 찾기
                cafe_content_element = element.find_element(By.CSS_SELECTOR, 'p.text')
                content = cafe_content_element.text

                # span 태그의 클래스명이 'date'인 요소 찾기
                date_element = element.find_element(By.CSS_SELECTOR, 'span.date')
                date_text = date_element.text
                
                # 날짜 처리
                if "분 전" in date_text:
                    minutes = int(date_text.split()[0])
                    post_date = datetime.now() - timedelta(minutes=minutes)
                    post_date_str = post_date.strftime("%Y.%m.%d.")
                elif "시간 전" in date_text:
                    hours = int(date_text.split()[0])
                    post_date = datetime.now() - timedelta(hours=hours)
                    post_date_str = post_date.strftime("%Y.%m.%d.")
                else:
                    # yyyy.mm.dd. 형식이면 그대로 사용
                    post_date_str = date_text

                # 모든 필드가 존재할 경우에만 MongoDB에 데이터 적재
                if title and url and cafe_name and content and post_date_str:
                    data = {
                        "제목": title,
                        "URL": url,
                        "카페이름": cafe_name,
                        "카페요약": content,
                        "업로드 날짜": post_date_str,
                        "검색키워드": "부산원아시아페스티벌"  # 검색 키워드를 지정 
                    }
                    all_data.append(data)

            except NoSuchElementException as e:
                print(f"Element not found: {e}")
            except Exception as e:
                print(f"Error processing element: {e}")

        # 페이지 이동
        if current_page % 10 == 0:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.btn.type_next")))
            next_button.click()
        else:
            next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//button[@class='btn number' and normalize-space(text())='{current_page + 1}']")))
            next_page_button.click()

        current_page += 1
        time.sleep(2)  # 페이지 로딩 대기

except Exception as e:
    print(f"Error occurred: {e}")

    # 데이터 적재 진행 
    for data in tqdm(all_data, desc='MongoDB에 데이터 적재 중'):
        collection.insert_one(data)

finally:
    driver.quit()

Error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF64CB7EEA2+31554]
	(No symbol) [0x00007FF64CAF7ED9]
	(No symbol) [0x00007FF64C9B872A]
	(No symbol) [0x00007FF64CA08434]
	(No symbol) [0x00007FF64CA0853C]
	(No symbol) [0x00007FF64CA4F6A7]
	(No symbol) [0x00007FF64CA2D06F]
	(No symbol) [0x00007FF64CA4C977]
	(No symbol) [0x00007FF64CA2CDD3]
	(No symbol) [0x00007FF64C9FA33B]
	(No symbol) [0x00007FF64C9FAED1]
	GetHandleVerifier [0x00007FF64CE88B1D+3217341]
	GetHandleVerifier [0x00007FF64CED5AE3+3532675]
	GetHandleVerifier [0x00007FF64CECB0E0+3489152]
	GetHandleVerifier [0x00007FF64CC2E776+750614]
	(No symbol) [0x00007FF64CB0375F]
	(No symbol) [0x00007FF64CAFEB14]
	(No symbol) [0x00007FF64CAFECA2]
	(No symbol) [0x00007FF64CAEE16F]
	BaseThreadInitThunk [0x00007FF8CDF1257D+29]
	RtlUserThreadStart [0x00007FF8CF60AF28+40]



MongoDB에 데이터 적재 중: 100%|██████████| 3936/3936 [00:34<00:00, 115.72it/s]
